# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
output_data_file = "../WeatherPy/output_data/city_weather.csv"
df = pd.read_csv(output_data_file)
weather_data = df.drop(columns=["Unnamed: 0"])
weather_data

City  Cloudiness Country        Date  Humidity    Lat     Lng  \
0         barrow           1      US  1586616104        82  71.29 -156.79   
1    east london         100      ZA  1586616104        78 -33.02   27.91   
2      inhambane         100      MZ  1586616105        68 -23.86   35.38   
3     nikolskoye          20      RU  1586616105        69  59.70   30.79   
4          bluff         100      NZ  1586615806        70 -46.60  168.33   
..           ...         ...     ...         ...       ...    ...     ...   
552        tahta           0      EG  1586616200        18  26.77   31.50   
553       carmen          20      MX  1586616113        74  18.63  -91.83   
554   caluquembe          36      AO  1586616200        51 -13.78   14.68   
555          wau         100      SS  1586616200        42   7.70   28.00   
556    vincennes          41      FR  1586616201        25  48.85    2.43   

     Max Temp  Windspeed  
0      -20.20      11.41  
1       73.40       3.36  
2       77.16       8.52  
3       39.20      17.90  
4       54.00       1.99  
..        ...        ...  
552     76.75      14.88  
553     82.40      11.41  
554     71.10       1.92  
555     88.48       2.46  
556     79.00       5.82  

[557 rows x 9 columns]

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
locations = weather_data[["Lat", "Lng"]]
humidity = weather_data['Humidity'].astype(float)

In [19]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [49]:
sunny_cities = weather_data.loc[(weather_data["Max Temp"] < 90) & (weather_data["Max Temp"] > 75) & (weather_data['Windspeed'] <= 10) & (weather_data['Cloudiness'] == 0)]
sunny_cities = sunny_cities.dropna()
#narrowed_city_df = narrowed_city_df.reset_index()
sunny_cities

City  Cloudiness Country        Date  Humidity    Lat    Lng  \
141          beroroha           0      MG  1586616125        43 -21.67  45.17   
146    kulhudhuffushi           0      MV  1586616125        66   6.62  73.07   
187  diego de almagro           0      CL  1586616131        20 -26.37 -70.05   
223     narasannapeta           0      IN  1586616135        80  18.42  84.05   
288         dhangadhi           0      NP  1586616148        17  28.71  80.60   
389            gwanda           0      ZW  1586616168        23 -20.93  29.00   
420            muleba           0      TZ  1586616172        48  -1.84  31.65   
436          challans           0      FR  1586616175        34  46.84  -1.87   
472         myitkyina           0      MM  1586616182        27  25.38  97.40   
526            matara           0      LK  1586616194        85   5.95  80.54   
530              bose           0      IT  1586616195        17  45.46   8.01   

     Max Temp  Windspeed  
141     82.85       3.87  
146     86.07       7.72  
187     76.41       6.55  
223     80.96       5.01  
288     83.59       4.45  
389     82.26       7.81  
420     75.85       5.08  
436     78.80       3.36  
472     77.13       3.00  
526     79.23       5.61  
530     80.01       4.70

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [50]:
sunny_cities['Nearest Hotel Name'] = ''
hotel_df = sunny_cities
hotel_df.head()

City  Cloudiness Country        Date  Humidity    Lat    Lng  \
141          beroroha           0      MG  1586616125        43 -21.67  45.17   
146    kulhudhuffushi           0      MV  1586616125        66   6.62  73.07   
187  diego de almagro           0      CL  1586616131        20 -26.37 -70.05   
223     narasannapeta           0      IN  1586616135        80  18.42  84.05   
288         dhangadhi           0      NP  1586616148        17  28.71  80.60   

     Max Temp  Windspeed Nearest Hotel Name  
141     82.85       3.87                     
146     86.07       7.72                     
187     76.41       6.55                     
223     80.96       5.01                     
288     83.59       4.45

In [51]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}

for index, row in hotel_df.iterrows(): 
    lat = row["Lat"]
    lng = row["Lng"]
    params['location'] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    try:
        response = requests.get(base_url, params=params).json()
        results = response['results']
        hotel_df.loc[index, 'Nearest Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print("City not found. Skipping...")
        
hotel_df['Nearest Hotel Name'] = hotel_df['Nearest Hotel Name'].replace('', np.nan)
hotel_df = hotel_df.dropna()
               
hotel_df

City not found. Skipping...


City  Cloudiness Country        Date  Humidity    Lat    Lng  \
146    kulhudhuffushi           0      MV  1586616125        66   6.62  73.07   
187  diego de almagro           0      CL  1586616131        20 -26.37 -70.05   
223     narasannapeta           0      IN  1586616135        80  18.42  84.05   
288         dhangadhi           0      NP  1586616148        17  28.71  80.60   
389            gwanda           0      ZW  1586616168        23 -20.93  29.00   
420            muleba           0      TZ  1586616172        48  -1.84  31.65   
436          challans           0      FR  1586616175        34  46.84  -1.87   
472         myitkyina           0      MM  1586616182        27  25.38  97.40   
526            matara           0      LK  1586616194        85   5.95  80.54   
530              bose           0      IT  1586616195        17  45.46   8.01   

     Max Temp  Windspeed       Nearest Hotel Name  
146     86.07       7.72      Haajy Guest House 1  
187     76.41       6.55  hotel luna del desierto  
223     80.96       5.01         G.lingaraju home  
288     83.59       4.45            Hotel Devotee  
389     82.26       7.81           Mafax Homeways  
420     75.85       5.08            NALPHIN HOTEL  
436     78.80       3.36      Citotel Le Challans  
472     77.13       3.00          Shwe Phyu Hotel  
526     79.23       5.61   Amaloh Boutique Resort  
530     80.01       4.70     Locanda Borgo Antico

In [52]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [41]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))